In [1]:
import os
import sys
import json
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv1D, MaxPooling1D
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

Using TensorFlow backend.


Couldn't import dot_parser, loading of dot files will not be possible.


In [2]:
from keras import backend as K
from sklearn.metrics import precision_recall_fscore_support

In [3]:
with open("pos_amazon_cell_phone_reviews.json") as f:
    data = json.load(f)
data = data['root']
#print(len(data))
df_pos = pd.DataFrame(data)
summaries_positive = []
text_positive = []
for i in data:
    summaries_positive.append(i['summary'])
    text_positive.append(i['text'])
label = [1] * 108664
df_pos['label'] = label

In [4]:
df_pos2 = df_pos[0:20000]

In [5]:
with open("neg_amazon_cell_phone_reviews.json") as f:
    data = json.load(f)
summaries_negative=[]
text_negative=[]
data = data['root']
df_neg = pd.DataFrame(data)
for i in data:
    summaries_negative.append(i['summary'])
    text_negative.append(i['text'])
label = [0] * len(data)
df_neg['label'] = label

In [6]:
df_neg2 = df_neg[:]

In [7]:
df = df_pos2.append(df_neg2)

In [8]:
seed = 123

In [9]:
X = df.iloc[:,:-1]
Y = df.iloc[:,-1]
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=seed)
#X_train

In [10]:
tokenizer = Tokenizer()

In [11]:
summary = list(X_train['summary'])
summary_test = list(X_test['summary'])

In [12]:
tokenizer.fit_on_texts(summary)
vocab = tokenizer.word_index
vocab_counts = tokenizer.word_counts

In [13]:
sequences = tokenizer.texts_to_sequences(summary)
sequences_test = tokenizer.texts_to_sequences(summary_test)

In [14]:
sequences_test.index(max(sequences_test,key = lambda x : len(x)))
len(max(sequences_test,key = lambda x : len(x)))

25

In [15]:
len(vocab)

6321

In [16]:
pad_length = len(max(sequences, key = lambda x: len(x)))
#pad_length = 22

In [17]:
pad_length

31

In [18]:
new_X_train = pad_sequences(sequences, maxlen = pad_length)
new_X_test =  pad_sequences(sequences_test, maxlen = pad_length)

In [19]:
new_X_train

array([[  0,   0,   0, ...,   0,  16, 220],
       [  0,   0,   0, ...,   0,   0,  45],
       [  0,   0,   0, ...,   0,  42, 114],
       ..., 
       [  0,   0,   0, ...,   0,   0,   1],
       [  0,   0,   0, ...,  34,  34,  34],
       [  0,   0,   0, ...,  36, 149, 163]])

In [20]:
y_train.values

array([1, 1, 0, ..., 1, 0, 1], dtype=int64)

In [21]:
new_X_test

array([[   0,    0,    0, ...,    0,    1,   54],
       [   0,    0,    0, ...,    0, 1346,  123],
       [   0,    0,    0, ...,    0,    1,   13],
       ..., 
       [   0,    0,    0, ...,   36,    4,   26],
       [   0,    0,    0, ...,   65,   47,  103],
       [   0,    0,    0, ...,    0,   72,  437]])

In [22]:
new_Y_train = to_categorical(y_train)
new_Y_train
#len(vocab)
len(new_Y_train)

26623

In [23]:
new_Y_test = to_categorical(y_test)
len(new_Y_test)

6656

In [104]:
BASE_DIR = ''

GLOVE_DIR = os.path.join(BASE_DIR, 'glove.6B')

In [105]:
embeddings_index = {}

f = open(os.path.join(GLOVE_DIR, 'glove.6B.100d.txt'),encoding='utf8')

for line in f:

    values = line.split()

    word = values[0]

    coefs = np.asarray(values[1:], dtype='float32')

    embeddings_index[word] = coefs

f.close()



In [106]:
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [108]:
# prepare embedding matrix
MAX_NB_WORDS = 5000
num_words = min(MAX_NB_WORDS, len(vocab) + 1)
EMBEDDING_DIM = 100
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))

for word, i in vocab.items():

    if i >= MAX_NB_WORDS:

        continue

    embedding_vector = embeddings_index.get(word)

    if embedding_vector is not None:

        # words not found in embedding index will be all-zeros.

        embedding_matrix[i] = embedding_vector

In [112]:
num_words

3427

In [24]:
def precision(y_true, y_pred):
    """Precision metric.
 
    Only computes a batch-wise average of precision.
 
    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
 
def recall(y_true, y_pred):
    """Recall metric.
    Only computes a batch-wise average of recall.
    Computes the recall, a metric for multi-label classification of
    how many relevant items are selected.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

MODEL

In [35]:
model = Sequential()
model.add(Embedding(len(vocab) + 1, 100, input_length=pad_length))
model.add(LSTM(256,dropout=0.5,recurrent_dropout=0.2))
#model.add(LSTM(128,dropout=0.2,return_sequences=False))
model.add(Dense(2,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', precision, recall])

In [36]:
history = model.fit(new_X_train, new_Y_train, validation_split = 0.20, epochs = 10)

Train on 21298 samples, validate on 5325 samples
Epoch 1/10
21298/21298 [==============================] - 157s - loss: 0.3208 - acc: 0.8570 - precision: 0.8575 - recall: 0.8568 - val_loss: 0.2250 - val_acc: 0.9099 - val_precision: 0.9096 - val_recall: 0.9102
Epoch 2/10
21298/21298 [==============================] - 153s - loss: 0.1816 - acc: 0.9340 - precision: 0.9341 - recall: 0.9340 - val_loss: 0.2127 - val_acc: 0.9139 - val_precision: 0.9140 - val_recall: 0.9138
Epoch 3/10
21298/21298 [==============================] - 148s - loss: 0.1459 - acc: 0.9463 - precision: 0.9466 - recall: 0.9460 - val_loss: 0.2241 - val_acc: 0.9147 - val_precision: 0.9146 - val_recall: 0.9149
Epoch 4/10
21298/21298 [==============================] - 148s - loss: 0.1243 - acc: 0.9546 - precision: 0.9548 - recall: 0.9545 - val_loss: 0.2231 - val_acc: 0.9174 - val_precision: 0.9180 - val_recall: 0.9166
Epoch 5/10
21298/21298 [==============================] - 149s - loss: 0.1089 - acc: 0.9590 - precision: 0.

In [37]:
scores = model.evaluate(new_X_test, new_Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))
y_pred = model.predict(new_X_test)
#y_pred
#precision([np.argmax(x) for x in new_Y_test], [np.argmax(x) for x in y_pred])
[p, r, f, _] = precision_recall_fscore_support([np.argmax(x) for x in new_Y_test], [np.argmax(x) for x in y_pred], average='binary')
#[[int(x[0].round()), int(x[1].round())] for x in y_pred]
#new_Y_test
#[np.argmax(x) for x in y_pred]
print("Precision: %.2f%%" % (p*100))
print("Recall: %.2f%%" % (r*100))
print("F-score: %.2f%%" % (f*100))

Accuracy: 92.00%
Precision: 91.95%
Recall: 94.94%
F-score: 93.42%


In [98]:
review = "Not that great. Don't buy"
review = [review]
#print(review)
to_check = tokenizer.texts_to_sequences(review)
to_check = pad_sequences(to_check, maxlen=pad_length)
to_check

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         2, 92,  1, 44, 20]])

In [99]:
predict_class = model.predict(to_check)
print(predict_class)
label = np.argmax(predict_class) 
print("Class : ", label)

if(label == 1):
    print("Positive")
else:
    print("Negative")


[[ 0.99462402  0.00451099]]
Class :  0
Negative


In [100]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 22, 32)            109664    
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               82432     
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 258       
Total params: 192,354
Trainable params: 192,354
Non-trainable params: 0
_________________________________________________________________


In [155]:
# Convolution

kernel_size = 5
filters = 128
pool_size = 4
model = Sequential()
model.add(Embedding(len(vocab) + 1, 32, input_length=pad_length))
model.add(Dropout(0.25))
model.add(Conv1D(filters,
                 kernel_size,
                 padding='valid',
                 activation='relu'))
model.add(MaxPooling1D(pool_size=pool_size))

model.add(LSTM(128))
model.add(Dense(2,activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', precision, recall])

In [152]:
model.fit(new_X_train, new_Y_train, validation_split = 0.16, epochs = 5)

Train on 8064 samples, validate on 1536 samples
Epoch 1/5
8064/8064 [==============================] - 7s - loss: 0.6015 - acc: 0.6272 - precision: 0.6267 - recall: 0.6261 - val_loss: 0.5030 - val_acc: 0.7051 - val_precision: 0.7051 - val_recall: 0.7051
Epoch 2/5
8064/8064 [==============================] - 5s - loss: 0.4578 - acc: 0.7388 - precision: 0.7388 - recall: 0.7386 - val_loss: 0.4946 - val_acc: 0.7051 - val_precision: 0.7054 - val_recall: 0.7044
Epoch 3/5
8064/8064 [==============================] - 5s - loss: 0.4083 - acc: 0.7580 - precision: 0.7566 - recall: 0.7609 - val_loss: 0.5124 - val_acc: 0.7155 - val_precision: 0.7155 - val_recall: 0.7155
Epoch 4/5
8064/8064 [==============================] - 5s - loss: 0.3855 - acc: 0.7675 - precision: 0.7679 - recall: 0.7670 - val_loss: 0.5326 - val_acc: 0.7012 - val_precision: 0.7012 - val_recall: 0.7012
Epoch 5/5
8064/8064 [==============================] - 5s - loss: 0.3686 - acc: 0.7757 - precision: 0.7755 - recall: 0.7762 - va